Consigna
========

Es un pájaro? Es un avión? No! Es el tp final de IA de este año: **clasificar pájaros vs aviones** utilizando **redes neuronales**.

En este trabajo práctico vamos a utilizar redes neuronales para intentar clasificar imágenes, determinando si el objeto que se ve en la imagen es un pájaro o un avión. 
Para ello, ya contamos con un set de 10.000 imágenes clasificadas en las que estamos seguros de lo que se ve: o un pájaro o un avión. 
Con estos datos entrenaremos una red neuronal, que luego deberá ser capaz de clasificar correctamente imágenes similares.

En clases vamos a ver cómo construir una red neuronal básica, y en este notebook ya tienen el código necesario para leer los datos, mostrar las imágenes, y evaluar la exactitud de la red.
El trabajo de ustedes consistirá en escribir el fragmento de código que construye y entrena la red neuronal, de forma tal que las métricas indiquen un 80% o más de accuracy en el conjunto de **test**.

Como el resultado no es el mismo si corren varias veces el código, lo que deben lograr es una red que en **al menos 3 de 5 ejecuciones**, logre 80% o más de accuracy.
No es válido entregar código que alguna vez tuvimos suerte y llegó al 80%, pero que habitualmente no lo alcanza.

La entrega se realizará subiendo el archivo ``tp_final.ipynb`` modificado por ustedes, a la **raiz** del repositorio del grupo.

Imports y configs
=================

In [ ]:
#import os
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

# imports de libs que usamos para leer y procesar datos en masa
import numpy as np
import pandas as pd

# lib que usamos para mostrar las imágenes
import matplotlib.pyplot as plt

# libs que usamos para tareas generales de machine learning: separar conjuntos de datos, evaluar exactitud, etc
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# libs que usamos para construir y entrenar redes neuronales
from keras.models import Sequential
from keras.layers import Dense, Activation, Input, Dropout, Convolution2D, MaxPooling2D, Flatten

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline

In [ ]:
# algunas configuraciones generales respecto a las imagenes
picture_size = 32
channels = 'rgb'

# nombres de las columnas de datos que vamos a usar como "entradas"
input_columns = []
for color in channels:
    input_columns.extend(['%s%i' % (color, i) 
                          for i in range(picture_size ** 2)])

Datos
=====

Leemos las 10.000 imagenes, que están guardadas en un solo archivo y mostramos 5 filas de ejemplo. Para que esto funcione, deben bajar el archivo ``data_tp_final.csv.gz`` desde [este link](https://drive.google.com/open?id=0B5cCDhynqkqDT3BSXzV6ak5Sc1E) y ubicarlo en el mismo directorio que el notebook.

Cada fila es una imagen. Cada imagen tiene muchos pixeles. Y cada pixel tiene 3 valores: cuánto de rojo, cuánto de verde y cuánto de azul (estos son "canales" de color).

Eso quiere decir que si la imagen tiene 32 x 32 pixeles, y cada pixel tiene 3 valores, entonces tenemos 32 x 32 x 3 = 3072 columnas.

La última columna, 'label', es un 0 cuando la imagen contiene un pájaro, y 1 cuando contiene un avión.

In [ ]:
data = pd.read_csv('data_tp_final.csv.gz')
data.sample(5)

Estandarizamos los valores, para que vayan de 0 a 1. Esto puede demorar un rato. Al final mostramos la nueva tabla, estandarizada.

In [ ]:
def standarize_inputs(dataset):
    dataset[input_columns] = dataset[input_columns] / 255
    #for input_column in input_columns:
    #    dataset[input_column] = dataset[input_column] / 255

In [ ]:
standarize_inputs(data)

data.sample(5)

Esta función nos permite pasarle un conjunto de imagenes (filas de la tabla), y nos las dibuja. También admite usar alguna columna de la tabla como título para cada imagen

In [ ]:
def show_images(samples, title=None):
    for index, sample in samples.iterrows():
        if title is not None:
            plt.title(str(sample[title]))

        sample_as_grid = sample[input_columns].values.reshape(len(channels), picture_size, picture_size)
        sample_as_grid = np.transpose(sample_as_grid, (1, 2, 0))
        plt.axis('off')
        plt.imshow(sample_as_grid, interpolation='nearest')

        plt.show()

Mostramos algunas imagenes de ejemplo al azar. Pueden ustedes distinguir lo que hay en cada imagen? (si ejecutan esta celda muchas veces, van a ir viendo imágenes diferentes)

In [ ]:
show_images(data.sample(10), title='label')

Y finalmente, separamos los datos en dos conjuntos, vamos a utilizar uno para entrenar la red y el otro para testearla (y de esa forma asegurarnos de que no estamos sobreentrenando, evaluar cómo se va a comportar con casos que no usó para entrenar, etc).

In [ ]:
train, test = train_test_split(data, test_size=0.2)
sets = (
    ('train', train),
    ('test', test),
)

for set_name, set_data in sets:
    print '#' * 20, set_name, 'labels', '#' * 20
    print set_data.label.value_counts()
    print

Red neuronal
============

Esta primer función es una comodidad, para no tener que repetir este fragmento de código cada vez que queramos alimentar a la red neuronal con datos de entrada. Lo que hace la función, es quedarse solo con los valores de las columnas de entrada.

In [ ]:
def extract_inputs(dataset):
    return dataset[input_columns].values

Y aquí es donde ustedes tienen que completar su entrega:

In [ ]:
#model = None  # Crear la red aquí y guardarla en esta variable 

model = Sequential([
        Dense(64, input_shape=(picture_size ** 2 * len(channels),)),
        Activation('relu'),
        
        Dense(32),
        Activation('relu'),
        
        Dense(16),
        Activation('relu'),
        
        Dense(1),
        Activation('sigmoid'),
])

model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy',]
)

In [ ]:
# Entrenar la red aquí

model.fit(
    extract_inputs(train), 
    train.label.values,
    nb_epoch = 25,
    batch_size = 200
)

Una vez entrenada, pueden evaluar la performance de la red neuronal utilizando los dos conjuntos, con este fragmento de código.

El valor que tienen que lograr que llege a 80% (0.8) o más, es **accuracy** en el conjunto de **test**.

In [ ]:
for set_name, set_data in sets:
    labels = set_data.label.values
    predicted_labels = np.rint(model.predict(extract_inputs(set_data)))

    print '#' * 25, set_name, '#' * 25
    print 'accuracy', accuracy_score(labels, predicted_labels)
    print classification_report(labels, predicted_labels)
    

Finalmente, podemos clasificar todas las imágenes del conjunto de test, y mostrar algunos ejemplos de casos que anduvieron bien y casos que anduvieron mal.

El título en cada imagen es la salida de la red neuronal: más cerca de 1 significa avión, más cerca de 0 significa pájaro.

In [ ]:
test_with_predictions = test.copy()
test_with_predictions['prediction'] = model.predict(extract_inputs(test_with_predictions))
test_with_predictions['predicted_label'] = np.rint(test_with_predictions.prediction)
is_correct = test_with_predictions.label == test_with_predictions.predicted_label

Bien clasificados:

In [ ]:
show_images(test_with_predictions[is_correct].sample(5), title='prediction')

Mal clasificados:

In [ ]:
show_images(test_with_predictions[~is_correct].sample(5), title='prediction')

Y si quieren probar con otras imagenes que ustedes consigan, pueden usar esta otra función, pasándole una lista de paths de las imágenes (solo jpg) en su disco. Pero **antes**, tienen que instalar otro paquete de python:

    pip install pillow
    
Esto es opcional, para que se diviertan, por eso no estaba al inicio en la consigna. Puede que renieguen para instalar pillow. En linux, puede que necesiten antes instalar otra cosa antes para que pillow se pueda instalar correctamente: ``sudo apt-get install libjpeg-dev``

In [ ]:
from PIL import Image

def classify_pictures(pictures_paths):
    raw_pictures_data = []
    
    for picture_path in pictures_paths:
        picture = Image.open(picture_path)

        picture = picture.resize((picture_size, picture_size), Image.ANTIALIAS)
        picture_data = np.array(list(zip(*picture.getdata()))).reshape(len(input_columns))
    
        raw_pictures_data.append(picture_data)
    
    pictures_data = pd.DataFrame(raw_pictures_data, columns=input_columns)

    standarize_inputs(pictures_data)

    pictures_data['prediction'] = model.predict(extract_inputs(pictures_data))
    
    show_images(pictures_data, title='prediction')

In [ ]:
#classify_pictures([
#    '/home/fisa/Pictures/halcon.jpg', # es un pajaro?
#    '/home/fisa/Pictures/f22.jpg',  #  es un avion?
#    '/home/fisa/Pictures/superman.jpg',  # no! es superman
#    # etc...
#])